# Retriever


## Buat Haystack Pipeline untuk retrieve data dari mongodb atlas

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever
from haystack.utils import Secret
from haystack.components.routers import ConditionalRouter
from typing import List
from haystack.components.generators import OpenAIGenerator 

d:\Enviroments\anaconda3\envs\day45-haystack\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
connection_string = os.getenv("MONGO_CONNECTION_STRING")
OPENAI_APIKEY = os.getenv("OPENAI_API_KEY")

In [3]:
document_store_products = MongoDBAtlasDocumentStore(
    database_name="depato_store",
    collection_name="products",
    vector_search_index="vector_index",
    full_text_search_index="search_index",
    mongo_connection_string=Secret.from_token(os.getenv("MONGO_CONNECTION_STRING"))
)
document_store_common = MongoDBAtlasDocumentStore(
    database_name="depato_store",
    collection_name="common_information",
    vector_search_index="vector_index_common",
    full_text_search_index=None,
    mongo_connection_string=Secret.from_token(os.getenv("MONGO_CONNECTION_STRING"))
)

In [4]:
TEMPLATE_BUILDER_ROUTER = """
You are a routing classifier. Your task is to return ONLY ONE of these strings based on the user's query:
"products"
"common"
Rules:
- Return "products" if the query refers to product details, specifications, variants, stock, brand, price, warranty, or anything related to an item sold.
- Return "common" if the query refers to general information such as shipping, returns, refunds, order status, policies, payment, help, customer service, or store info.
Your output must be exactly one of:
"products"
"common"

User Query:
{{ query }}
"""


In [5]:
routes = [
    {
        "condition": "{{ replies[0] | trim | lower == 'products' }}", 
        "output": "{{ embedding }}",          
        "output_name": "products_route",       
        "output_type": List[float]              
    },
    {
        "condition": "{{ replies[0] | trim | lower == 'common' }}",  
        "output": "{{ embedding }}",
        "output_name": "common_route",
        "output_type": List[float]
    }
]

router = ConditionalRouter(routes)

In [6]:
llm = OpenAIGenerator(model="gpt-3.5-turbo",api_key=Secret.from_token(OPENAI_APIKEY)) 

In [7]:
pipeline = Pipeline()

pipeline.add_component("embedder", SentenceTransformersTextEmbedder())
pipeline.add_component("prompt_builder", PromptBuilder(template=TEMPLATE_BUILDER_ROUTER))
pipeline.add_component("decision", llm)
pipeline.add_component("router", router)
pipeline.add_component("retriever_common",
    MongoDBAtlasEmbeddingRetriever(document_store=document_store_common, top_k=3)
)
pipeline.add_component("retriever_products",
    MongoDBAtlasEmbeddingRetriever(document_store=document_store_products, top_k=10)
)


PromptBuilder has 1 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


In [8]:
pipeline.connect("embedder.embedding", "router.embedding")
pipeline.connect("prompt_builder", "decision")
pipeline.connect("decision.replies", "router.replies")
pipeline.connect("router.products_route", "retriever_products.query_embedding")
pipeline.connect("router.common_route", "retriever_common.query_embedding")


🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - prompt_builder: PromptBuilder
  - decision: OpenAIGenerator
  - router: ConditionalRouter
  - retriever_common: MongoDBAtlasEmbeddingRetriever
  - retriever_products: MongoDBAtlasEmbeddingRetriever
🛤️ Connections
  - embedder.embedding -> router.embedding (list[float])
  - prompt_builder.prompt -> decision.prompt (str)
  - decision.replies -> router.replies (list[str])
  - router.products_route -> retriever_products.query_embedding (List[float])
  - router.common_route -> retriever_common.query_embedding (List[float])

In [9]:
user_query = "barang apa yang bagus?"
result = pipeline.run({
    "embedder": {"text": user_query},
    "prompt_builder": {"query": user_query},
})
result

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.30it/s]


{'decision': {'meta': [{'model': 'gpt-3.5-turbo-0125',
    'index': 0,
    'finish_reason': 'stop',
    'usage': {'completion_tokens': 1,
     'prompt_tokens': 131,
     'total_tokens': 132,
     'completion_tokens_details': {'accepted_prediction_tokens': 0,
      'audio_tokens': 0,
      'reasoning_tokens': 0,
      'rejected_prediction_tokens': 0},
     'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}]},
 'retriever_products': {'documents': [Document(id=882dc9a29c19a5f6dba5e44d6389405be3a567dd104697bc0b885bc0d7697f2c, content: 'Billabong Boys' Kids Permahang 10 Short Sleeve T-Shirt
    Premium 26/1, 160 gram t-shirt with soft han...', meta: {'asin': 'B01AH877BE', 'title': "Billabong Boys' Kids Permahang 10 Short Sleeve T-Shirt", 'brand': nan, 'price': 15.94, 'gender': 'male', 'material': 'Cotton', 'category': 'Tops'}, score: 0.6685053706169128, embedding: vector of size 768),
   Document(id=97edda2596d9a97846bc490f4ee92df4864ef8362620938ae4ec555c19a4a780, content: '

In [10]:
user_query = "Kapan barang dikirim?"
result = pipeline.run({
    "embedder": {"text": user_query},
    "prompt_builder": {"query": user_query},
})
docs = result["retriever_common"]["documents"]
for doc in docs:
    print(doc.content, "\n")

Batches: 100%|██████████| 1/1 [00:00<00:00, 13.04it/s]


dan produk yang
    sudah digunakan tidak dapat dikembalikan.
-   Processing Time: Proses refund memakan waktu 3–5 hari kerja setelah
    barang diterima dan diverifikasi.

2.  Shipping Information

-   Shipping Partners: JNE, J&T Express, SiCepat, Pos Indonesia, Ninja
    Xpress.
-   Estimated Delivery Time: Jabodetabek: 1–3 hari kerja, Luar Jawa: 3–7
    hari kerja.
-   Shipping Fee: Dihitung otomatis berdasarkan berat dan lokasi
    pengiriman.
-   Tracking: Nomor resi akan dikirim melalui email/SMS setelah pesanan
    dikonfirmasi.
-   International Shipping: Tersedia untuk  

   pengiriman, dan komunikasi pelanggan.
-   Security: Informasi pribadi dilindungi dengan enkripsi SSL.
-   Third-Party Sharing: Kami tidak menjual data pelanggan kepada pihak
    ketiga.

4.  Payment Information

-   Available Methods: Transfer bank, kartu kredit/debit, e-wallet
    (GoPay, OVO, DANA, ShopeePay), dan COD (Cash on Delivery).
-   Payment Confirmation: Pembayaran otomatis terverifikasi dalam 5

## Menggunakan Meta Data untuk memfilter

### Single Filter

In [35]:
filters = {
    "field":"meta.category","operator":"==","value":"Dresses/Jumpsuits"
}

In [37]:
pipeline.run(
    {
        "embedder":{
            "text":"comfortable dress for going out with friends"
        },
        "router":{
            "target": "products"
        },
        "retriever_products":{
            "filters": filters
        }
    }
)

Batches: 100%|██████████| 1/1 [00:00<00:00, 28.34it/s]


{'retriever_products': {'documents': [Document(id=1e51b2fdd4606b16cfb582e64ffd7fb44e36fa2423fed1cf0f02ad5396015c3f, content: 'Rasta Imposta Flag Dress USA
    The usa flag dress is the best outfit to wear if you want to show off ...', meta: {'asin': 'B00C3DLHY0', 'title': 'Rasta Imposta Flag Dress USA', 'brand': 'Rasta Imposta', 'price': 24.33, 'gender': 'female', 'material': 'Polyester', 'category': 'Dresses/Jumpsuits'}, score: 0.7217811346054077, embedding: vector of size 768),
   Document(id=13a473e9ef572dbfe8f37cef1f1b59e02d43394e90addbe598c83ce4560117fd, content: 'Ivanka Trump Women's Lace Dress
    Sleeveless floral dress with blush underlay perfect to we at for a ...', meta: {'asin': 'B01FH59HXS', 'title': "Ivanka Trump Women's Lace Dress", 'brand': nan, 'price': 98.6, 'gender': 'female', 'material': 'Polyester', 'category': 'Dresses/Jumpsuits'}, score: 0.7054649591445923, embedding: vector of size 768),
   Document(id=c72b53a22948126b5860fd58d59593a09e23dbca93666fcb36e6c4b60510

### Multiple Filter

In [38]:
filters = {
    "operator":"AND",
    "conditions":[
        {"field":"meta.category","operator":"==","value":"Dresses/Jumpsuits"},
        {"field":"meta.price", "operator":"<=", "value":50},
    ]
}

In [41]:
pipeline.run(
    {
        "embedder":{
            "text":"comfortable dress for going out with friends"
        },
        "router":{
            "target":"products"
        },
        "retriever_products":{
            "filters": filters
        }
    }
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 14.37it/s]


{'retriever_products': {'documents': [Document(id=1e51b2fdd4606b16cfb582e64ffd7fb44e36fa2423fed1cf0f02ad5396015c3f, content: 'Rasta Imposta Flag Dress USA
    The usa flag dress is the best outfit to wear if you want to show off ...', meta: {'asin': 'B00C3DLHY0', 'title': 'Rasta Imposta Flag Dress USA', 'brand': 'Rasta Imposta', 'price': 24.33, 'gender': 'female', 'material': 'Polyester', 'category': 'Dresses/Jumpsuits'}, score: 0.7217811346054077, embedding: vector of size 768),
   Document(id=a5c3efc5fa4f4c86d8c81dabb74b64c03020687d84be462053aaefc2f94d58cb, content: 'A. Byer Juniors' Cable-Knit Fashion Sweater Dress
    Soft knit styling sweater dress with deep cowl ne...', meta: {'asin': 'B011T4NTU8', 'title': "A. Byer Juniors' Cable-Knit Fashion Sweater Dress", 'brand': nan, 'price': 34.99, 'gender': 'female', 'material': 'Polyester', 'category': 'Dresses/Jumpsuits'}, score: 0.6889417767524719, embedding: vector of size 768),
   Document(id=db38bf21b88afde79cc0300a25915261b8d75f020

### Advanced Filtering

In [42]:
filters = {
    "operator":"AND",
    "conditions":[
        {"field":"meta.category","operator":"==","value":"Dresses/Jumpsuits"},
        {"field":"meta.price", "operator":"<=", "value":50},
        {
            "operator":"OR",
            "conditions":[
                {"field":"meta.material","operator":"==","value":"Cotton"},
                {"field":"meta.material","operator":"==","value":"Polyester"}
            ]
        }
    ]
    
}

In [44]:
pipeline.run(
    {
        "embedder":{
            "text":"comfortable dress for going out with friends"
        },
        "router":{
            "target":"products"
        },
        "retriever_products":{
            "filters": filters
        }
    }
)

Batches: 100%|██████████| 1/1 [00:00<00:00, 10.81it/s]


{'retriever_products': {'documents': [Document(id=1e51b2fdd4606b16cfb582e64ffd7fb44e36fa2423fed1cf0f02ad5396015c3f, content: 'Rasta Imposta Flag Dress USA
    The usa flag dress is the best outfit to wear if you want to show off ...', meta: {'asin': 'B00C3DLHY0', 'title': 'Rasta Imposta Flag Dress USA', 'brand': 'Rasta Imposta', 'price': 24.33, 'gender': 'female', 'material': 'Polyester', 'category': 'Dresses/Jumpsuits'}, score: 0.7217811346054077, embedding: vector of size 768),
   Document(id=a5c3efc5fa4f4c86d8c81dabb74b64c03020687d84be462053aaefc2f94d58cb, content: 'A. Byer Juniors' Cable-Knit Fashion Sweater Dress
    Soft knit styling sweater dress with deep cowl ne...', meta: {'asin': 'B011T4NTU8', 'title': "A. Byer Juniors' Cable-Knit Fashion Sweater Dress", 'brand': nan, 'price': 34.99, 'gender': 'female', 'material': 'Polyester', 'category': 'Dresses/Jumpsuits'}, score: 0.6889417767524719, embedding: vector of size 768),
   Document(id=db38bf21b88afde79cc0300a25915261b8d75f020